# Configurations

In [38]:
import config_file;
configs = config_file.readConfigs();
configs['camera_url']

'http://192.168.0.103:8080/shot.jpg'

# Main Classes

In [39]:
import numpy as np;
import pandas as pd;
import cv2;
import math;
from tqdm.notebook import tqdm;
from time import sleep;
import termcolor;
from ipdb import set_trace;
import PIL;
import IPython;
from io import BytesIO;

class DatabaseLogger():

    def log(self, people):
        data = pd.DataFrame(columns=['']);

In [ ]:
class VideoStreamBase():

    def __del__(self):
        cv2.VideoCapture(0).release();
        cv2.destroyAllWindows();

    def getImage(self) -> np.ndarray:
        raise NotImplementedError();

    def displayVideo(self, limit: int = 0, lag: int = 0, display_id: int = 1, width: int = 400, height: int = 200):
        try:
            while(self.displayImage(display_id, width, height)):
                sleep(lag);
        except(KeyboardInterrupt):
            print(termcolor.colored('video stream stopped voluntarily', 'green'));

    def displayImage(self, display_id: int = None, width: int = 400, height: int = 200, img = None) -> bool:
        img = self.getImage() if img is None else img;
        if(img is None): return False;

        img = PIL.Image.fromarray(img, 'RGB')
        buffer = BytesIO()
        img.save(buffer, format="jpeg")
        display(
            IPython.display.Image(
                data=buffer.getvalue(), format='jpeg', width=width, height=height
                ), display_id=display_id);

        return True;

    def __len__(self):
        raise NotImplementedError();

    def fps(self):
        raise NotImplementedError();

    def download(self, path: str = './output.mp4', fps: float = None, limit: int = None, display: bool = False,
                 safeMode: bool = True, width: int = None, height: int = None):

        nFrame = 1 if len(self) == 1 else (len(self) - 1 if limit is None else limit);
        if(nFrame == 0): raise Exception('No Frames Found');

        img = self.getImage();

        if(height is None): height = img.shape[0];
        if(width  is None): width  = img.shape[1];

        fourcc =  cv2.VideoWriter_fourcc(*'XVID');
        fps_ = self.fps() if fps is None else fps;
        outputVideo = cv2.VideoWriter(path, fourcc, fps_, (width, height), 1)

        # set_trace();
        with tqdm(total=nFrame) as progressbar:
            currentFrame = 0;
            nextImg = None;
            while(currentFrame < nFrame and img is not None):
                try:
                    if(display): self.displayImage(1, width, height, img);
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB);
                    nextImg = self.getImage();
                except Exception as e:
                    if(safeMode):
                        print(termcolor.colored('Exception:', 'red'));
                        print(e);
                    else:
                        raise e;

                outputVideo.write(img);
                currentFrame += 1;
                progressbar.update(1);
                img = nextImg;

        outputVideo.release();

In [40]:
import face_recognition;
import os;
import json;


class FaceDetector(VideoStreamBase):

    video: VideoStreamBase

    def __len__(self):
        return len(self.video);

    def fps(self):
        return self.video.fps();

    def __init__(self, video: VideoStreamBase):
        self.video = video;

        self.known_faces = None;
        with open(configs['db_face'], 'r') as f:
            embeddings = json.load(f);

            self.known_faces = [None] * len(embeddings);
            for el in embeddings:
                self.known_faces[el['id']] = np.array(el['embedding']);

        self.known_names = None;
        with open(configs['db_person'], 'r') as f:
            people = json.load(f);

            self.known_names = [None] * len(people);
            assert(len(self.known_names) == len(self.known_faces))

            for el in people:
                self.known_names[el['face']] = (el['id'], el['name']);

    def getImage(self):
        img = self.video.getImage();
        if(img is None): return None;

        face_locations = face_recognition.face_locations(img, model=configs['face_locations_model'])
        face_encodings = face_recognition.face_encodings(img, face_locations, model=configs['face_encodings_model'])
        # set_trace();

        people = [];
        for face_encoding in face_encodings:

            # print(type(face_encoding), face_encoding);
            # print(type(self.known_faces[0]), self.known_faces[0]);

            tolerance = 0.60;
            # match = face_recognition.compare_faces(self.known_faces, face_encoding, tolerance=tolerance)
            match = face_recognition.face_distance(self.known_faces, face_encoding)

            # id = np.argmax(match);
            # name = self.known_names[id] if match[id] else None;

            id = np.argmin(match);
            personId, name = (self.known_names[id]) if (match[id] <= tolerance) else (None, None);

            people.append((personId, name));

        for (top, right, bottom, left), name in zip(face_locations, people):
            color = (255, 0, 0) if name is None else (0, 255, 0);
            cv2.rectangle(img, (left, top), (right, bottom), color, 2);

            if(name is None): continue;

            cv2.rectangle(img, (left, bottom - 42), (right, bottom), color, cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(img, name, (left + 5, bottom - 8), font, 1.5, (0, 0, 0), 2);

        return img;

# video = FaceDetector(VideoReader(videoFile), facesDir);
# video.download(outputFile, 25);

# Main Classes for Local Setting

In [41]:
import numpy as np;
import cv2;
import termcolor;


class LocalVideoStream(VideoStreamBase):

    def displayVideo(self, fps = 30, displayName = 'Streaming'):
        while(True):
            img = self.getImage();
            if(img is None): break;

            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR);
            cv2.imshow(displayName, img);

            if(cv2.waitKey(1) & 0xFF == ord('q')): # press `q` to break
                break;


        print(termcolor.colored('video stream stopped voluntarily', 'green'));
        print(termcolor.colored('WARNING: object deleted', 'yellow'));
#         self.__del__();
        super(LocalVideoStream, self).__del__();


In [42]:
class LocalFaceDetector(LocalVideoStream, FaceDetector):
    def __init__(self, video: VideoStreamBase):
        super(LocalFaceDetector, self).__init__(video);


# Offilne Setting

In [43]:
import numpy as np
import cv2
import IPython
import PIL


class VideoReader(LocalVideoStream):

    def __init__(self, file):
        self.capture = cv2.VideoCapture(file)

    def __del__(self):
        self.capture.release()
        super(VideoReader, self).__del__();

    def __len__(self):
        return int(self.capture.get(cv2.CAP_PROP_FRAME_COUNT));

    def fps(self):
        return self.capture.get(cv2.CAP_PROP_FPS);

    def getImage(self):
        ret, frame = self.capture.read()
        if(ret == False): return None;

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        return frame;

# video = VideoReader(configs['video_file']);
# video.displayVideo();

In [45]:
video = LocalFaceDetector(VideoReader(configs['video_file']));
video.download(configs['output_file'], limit=5);
